In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Number of Purchasers

In [2]:
purchase_data.head(5)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
print(f'The number of unique names is:  {purchase_data["SN"].nunique()}')
print(f'The number of names is:  {purchase_data["SN"].count()}')

The number of unique names is:  576
The number of names is:  780


In [4]:
# Determine the number of purchases that are repeat purchases
# what this does => The non-repeated SN are labeled false and the first duplicated value is labeled false; true is only the 2nd, 3rd, 4th, etc duplication/purchase
purchase_data[purchase_data["SN"].duplicated()].count()["SN"]

204

In [5]:
# Total purchases from repeat purchasers
purchase_data[purchase_data["SN"].duplicated(keep=False)]["SN"].count()
# Note:  unique purchases by repeat purchasers would be 366 - 204 = 162

366

In [6]:
# Determine the distribution of repeat purchasers
repeat_purchasers = purchase_data[purchase_data["SN"].duplicated(keep=False)]['SN'].value_counts()
print(f"Number of customers with 5 or more purchases:  {repeat_purchasers[repeat_purchasers >= 5].count()}")
print(f"Number of customers with 4 purchases:  {repeat_purchasers[repeat_purchasers == 4].count()}")
print(f"Number of customers with 3 purchases:  {repeat_purchasers[repeat_purchasers == 3].count()}")
print(f"Number of customers with 2 purchases:  {repeat_purchasers[repeat_purchasers == 2].count()}")
print(f"Number of customers multiple purchases:  {repeat_purchasers[repeat_purchasers > 1].count()}")

Number of customers with 5 or more purchases:  1
Number of customers with 4 purchases:  2
Number of customers with 3 purchases:  35
Number of customers with 2 purchases:  124
Number of customers multiple purchases:  162


## Analysis check
This can be confusing without defining it clearly:
  *  366 individual purchases by only people who were repeat buyers
  *  204 individual purchases that were 2nd, 3rd, 4th, or more purchase
  *  162 individual purchases that were 1st purchase = this is equivalent to the number of purchasers and sum of value_counts

In [7]:
total_players = pd.DataFrame({"Total Players": [purchase_data["SN"].nunique()]})
total_players

,Total Players
0,576


## Purchasing Analysis (Total)

In [8]:
# Set simple df name for ease of remembering
df=purchase_data


#### Dataframe from Dictionary of Lists (aka [Scalars])

In [9]:
# Create dictionary holding values
# Find number of unique items, average price, and other summary information
summary_dict={}
summary_dict["Unique Items"] = df["Item ID"].nunique()
summary_dict["Average Price"] = "${0:,.2f}".format(df["Price"].mean()) 
summary_dict["Number of Purhases"] = df["Purchase ID"].count()
summary_dict["Total Revenue"] = "${0:,.2f}".format(df["Price"].sum())


In [10]:
# check structure
summary_dict

{'Unique Items': 183,
 'Average Price': '$3.05',
 'Number of Purhases': 780,
 'Total Revenue': '$2,379.77'}

In [11]:
# Generate summary dataframe; must put dictionary into a list or set index like `index=[0]` as second paramter
summary = pd.DataFrame([summary_dict])
summary

,Unique Items,Average Price,Number of Purhases,Total Revenue
0,183,$3.05,780,"$2,379.77"


#### Alternate Method - Dataframe from Dictionary of Lists

In [12]:
# Create dictionary holding values 
summary_dict["Unique Items"] = [df["Item ID"].nunique()]
summary_dict["Average Price"] = ["${0:,.2f}".format(df["Price"].mean())] 
summary_dict["Number of Purhases"] = [df["Purchase ID"].count()]
summary_dict["Total Revenue"] = ["${0:,.2f}".format(df["Price"].sum())]

In [13]:
# Generate summary dataframe
summary = pd.DataFrame(summary_dict)
summary

,Unique Items,Average Price,Number of Purhases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

In [14]:
# find percentage and count of male and female and other shoppers
df_unique_people = df.drop_duplicates("SN", keep="first")

# Create groupby
gender_df = df_unique_people[["Gender", "Age", "Price"]].groupby("Gender")

# Find total number of people broken down by female, male and other
gender_counts = gender_df.count()

# Find total number of people (female+male+other)
gender_total = gender_counts.sum()

# Calculate percentage of population
gender_counts["Price"]= gender_counts/gender_total*100

# format results
gender_counts["Price"] = gender_counts["Price"].map('{0:,.2f}%'.format)

# rename dataframe columns
gender_counts.columns=["Total Count", "Percentage of Shoppers"]

# view results
gender_counts.head()


,Total Count,Percentage of Shoppers
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [15]:
# Remind myself of df columns
df.head(5)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [16]:
# May want to use df_drop instead of df so it does not have repeated SN

# Collect necessary columns and groupby gender
purchases_df = df[["Price", "Item ID", "Gender"]].groupby("Gender")

# Calculate Price sum, # records, and mean
df_purchases = purchases_df.agg({"Price": [np.sum, np.count_nonzero, np.mean]})

# Add column of total purchases revenue devided by total purchases by gender
df_purchases["purchases_per_person"] = df_purchases.loc[:,("Price", "sum")]/gender_counts["Total Count"]

# reduce multi-index
df_purchases.columns = df_purchases.columns.to_flat_index()

# Rename and specify columns
df_purchases.columns=["Total Purchase Value", "Purchase Count", "Average Purchase Price", "Avg Total Purchases per Person"]
df_purchases=df_purchases[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchases per Person"]]

# Format columns
df_purchases["Average Purchase Price"]=df_purchases["Average Purchase Price"].map('${0:,.2f}'.format)
df_purchases["Total Purchase Value"]=df_purchases["Total Purchase Value"].map('${0:,.2f}'.format)
df_purchases["Purchase Count"]=df_purchases["Purchase Count"].map('{0:,.0f}'.format)
df_purchases["Avg Total Purchases per Person"]=df_purchases["Avg Total Purchases per Person"].map('${0:,.2f}'.format)

df_purchases

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchases per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [17]:
# review dataframe
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [18]:
# create binning limits and labels
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]
bin_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# # allow pandas to make a copy.  Not sure I didn't use .copy()
df_age=df.copy()

In [19]:
# add new column based on categories created with bins and bin_labels
df_age["Age Groups"] = pd.cut(df_age['Age'], bins, labels=bin_labels, right=False)

# check calculations
df_age[df_age["Age"]<=10]

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Groups
27,27,Eusri44,7,Male,96,Blood-Forged Skeletal Spine,3.09,<10
32,32,Aeralria27,10,Male,133,Faith's Scimitar,4.09,10-14
33,33,Haillyrgue51,7,Male,44,Bonecarvin Battle Axe,2.38,<10
37,37,Seuthep89,8,Male,73,Ritual Mace,2.05,<10
38,38,Reulae52,10,Female,116,Renewed Skeletal Katana,4.18,10-14
54,54,Zhisrisu83,10,Male,25,Hero Cane,4.35,10-14
78,78,Haillyrgue51,7,Male,50,Dawn,4.60,<10
81,81,Heudai45,8,Female,174,Primitive Blade,3.47,<10
141,141,Zhisrisu83,10,Male,60,Wolf,3.54,10-14
155,155,Aillyrin83,10,Female,98,"Deadline, Voice Of Subtlety",2.89,10-14


In [20]:
#  Groupby the binned data and get summary data of purchasers

#Number of purchasers by age group (as a series)
df_purchasers_by_age_group = df_age.groupby("Age Groups").count()['SN']

#Sum of purchass by age (as a series)
df_purchases_by_age_group = df_age.groupby("Age Groups").sum()['Price']

# Total number of purchasers across all age groups
df_purchasers_total = df_purchasers_by_age_group.sum()

# Total revenue across all age groups
df_purchases_total = df_purchases_by_age_group.sum()
print(f"Total Revenue: {df_purchases_total}")

# calculation of percent of total (Purchasers) (as a series)
df_percent_by_age_group = round(df_purchasers_by_age_group/df_purchasers_total*100,2)

# calculation of percent of total (Revenue) (as a series)
df_percent_revenue_by_age_group = round(df_purchases_by_age_group/df_purchases_total*100,2)

# Combine multiple series into a dataframe (must have axis=1)
df_age_summary = pd.concat([df_purchasers_by_age_group, df_purchases_by_age_group, df_percent_by_age_group, df_percent_revenue_by_age_group], axis=1)

# dataframe formatting
df_age_summary.columns = ["Total Purchasers", "Total Revenue", "Percent Total (Purchasers)", "Percent Total (Revenue)"]
df_age_summary["Percent Total (Purchasers)"] = df_age_summary["Percent Total (Purchasers)"].map('{0:,.2f}%'.format)
df_age_summary["Total Revenue"] = df_age_summary["Total Revenue"].map('${0:,.2f}'.format)
df_age_summary["Percent Total (Revenue)"] = df_age_summary["Percent Total (Revenue)"].map('{0:,.2f}%'.format)
df_age_summary

Total Revenue: 2379.7700000000004


,Total Purchasers,Total Revenue,Percent Total (Purchasers),Percent Total (Revenue)
Age Groups,,,,
<10,23,$77.13,2.95%,3.24%
10-14,28,$82.78,3.59%,3.48%
15-19,136,$412.89,17.44%,17.35%
20-24,365,"$1,114.06",46.79%,46.81%
25-29,101,$293.00,12.95%,12.31%
30-34,73,$214.00,9.36%,8.99%
35-39,41,$147.67,5.26%,6.21%
40+,13,$38.24,1.67%,1.61%


## Purchasing Analysis (Age)

In [21]:
df_purchasing_age = df.copy()

In [22]:
df_purchasing_age["Age Groups"] = pd.cut(df_purchasing_age['Age'], bins, labels=bin_labels, right=False)

In [23]:
df_purchasing_age[df_purchasing_age["Age"]<15]

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Groups
26,26,Lirtossa84,11,Male,71,Demise,1.61,10-14
27,27,Eusri44,7,Male,96,Blood-Forged Skeletal Spine,3.09,<10
32,32,Aeralria27,10,Male,133,Faith's Scimitar,4.09,10-14
33,33,Haillyrgue51,7,Male,44,Bonecarvin Battle Axe,2.38,<10
37,37,Seuthep89,8,Male,73,Ritual Mace,2.05,<10
38,38,Reulae52,10,Female,116,Renewed Skeletal Katana,4.18,10-14
54,54,Zhisrisu83,10,Male,25,Hero Cane,4.35,10-14
56,56,Raesty92,12,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,10-14
76,76,Mindadaran26,13,Female,88,"Emberling, Defender of Delusions",3.75,10-14
78,78,Haillyrgue51,7,Male,50,Dawn,4.60,<10


In [24]:
# Note:  Number of Purchases matches the number of unique Purchasers

# Collect lists
number_of_purchases = df_purchasing_age.groupby("Age Groups").count()["Purchase ID"]
average_price = df_purchasing_age.groupby("Age Groups").mean()["Price"]
revenue =  df_purchasing_age.groupby("Age Groups").sum()["Price"]
number_of_purchasers = df_purchasing_age.groupby("Age Groups").nunique()["SN"]

# Create dataframe from Dictionary of lists
purchase_summary_df = pd.DataFrame({"Total # Purchases":number_of_purchases, "Average Purchase Price":average_price, "Total Purchase Revenue":revenue, "Average Purchase Amount per Person": revenue/number_of_purchasers })

# add average number of purchases
purchase_summary_df["Average number of purchases"] = round(purchase_summary_df['Average Purchase Amount per Person']/purchase_summary_df["Average Purchase Price"],2)

# formatting
purchase_summary_df["Average Purchase Price"] = purchase_summary_df["Average Purchase Price"].map('${0:,.2f}'.format)
purchase_summary_df["Total Purchase Revenue"] = purchase_summary_df["Total Purchase Revenue"].map('${0:,.2f}'.format)
purchase_summary_df["Average Purchase Amount per Person"] = purchase_summary_df["Average Purchase Amount per Person"].map('${0:,.2f}'.format)
purchase_summary_df

,Total # Purchases,Average Purchase Price,Total Purchase Revenue,Average Purchase Amount per Person,Average number of purchases
Age Groups,,,,,
<10,23,$3.35,$77.13,$4.54,1.35
10-14,28,$2.96,$82.78,$3.76,1.27
15-19,136,$3.04,$412.89,$3.86,1.27
20-24,365,$3.05,"$1,114.06",$4.32,1.41
25-29,101,$2.90,$293.00,$3.81,1.31
30-34,73,$2.93,$214.00,$4.12,1.40
35-39,41,$3.60,$147.67,$4.76,1.32
40+,13,$2.94,$38.24,$3.19,1.08


## Top Spenders

In [25]:
total_purchases_top5 = df.groupby("SN").agg(['count','mean','sum'])["Price"].sort_values(by="sum", ascending=False).head(5)
total_purchases_top5.columns=["Total Purchases", "Average Purchase Cost", "Sum of All Purchases"]
total_purchases_top5["Sum of All Purchases"] = total_purchases_top5["Sum of All Purchases"].map('${0:,.2f}'.format)
total_purchases_top5["Average Purchase Cost"] = total_purchases_top5["Average Purchase Cost"].map('${0:,.2f}'.format)
total_purchases_top5

,Total Purchases,Average Purchase Cost,Sum of All Purchases
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [26]:
# get all three items at the beginning
popular_df = df[['Item ID', 'Item Name', 'Price']].groupby(["Item ID", "Item Name"]).agg(["count", "mean", "sum"])["Price"]
popular_df.reset_index(inplace=True)
popular_df.sort_values(by="count", ascending=False, inplace=True)
popular_df.set_index(["Item ID", "Item Name"], inplace=True)
popular_df.columns=["Total Purchases", "Item Price", "Sum of All Purchases"]
popular_df.head(5)

,,Total Purchases,Item Price,Sum of All Purchases
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


## Most Profitable Items

In [27]:
# get all three items at the beginning
profitable_df = df[['Item ID', 'Item Name', 'Price']].groupby(["Item ID", "Item Name"]).agg(["count", "mean", "sum"])["Price"]
profitable_df.reset_index(inplace=True)
profitable_df.sort_values(by="sum", ascending=False, inplace=True)
profitable_df.set_index(["Item ID", "Item Name"], inplace=True)
profitable_df.columns=["Total Purchases", "Item Price", "Sum of All Purchases"]
profitable_df.head(5)

,,Total Purchases,Item Price,Sum of All Purchases
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
